# Analisis exploratorio - Gestión de la Contratación Pública en los Establecimientos Carcelaríos en Colombia

Para fines de este proyecto y de acuerdo con lo que se ha mencionado en anteriores entregas, en principio se tomará los datos de la página del SECOP. En ella, es posible extraer la información relacionada con la gestión de la contratación pública relacionada con el INPEC. 

De acuerdo con lo anterior, filtrando en la base de datos por Nombre Entidad INPEC, es posible extraer un archivo tipo csv la cual contiene 72.520 filas y cuenta con 72 dimensiones. A continuación, se presenta el paso a paso de la limpieza de dichos datos.

Realizando un escaneo general de la forma de la base se encuentra que las dimensiones son: UID, año cargue SECOP, año firma contrato, nivel entidad, orden entidad, nombre entidad, NIT de la entidad, entre otras características relacionadas con los contratistas que han tenido algún contrato con el INPEC. Asimismo, es posible observar que también existen dimensiones relacionadas con la fecha inicio del contrato, fecha fin del contrato tiempo adiciones en meses, plazo de ejecución del contrato, cuantía del contrato, y valor total de adiciones relacionadas con las características del contrato.

**Objetivo:** Ofrecer información de valor al INPEC para la toma de decisiones que impacte positivamente en la administración de su contratación pública. 

**Requerimiento del cliente** 
* Análisis de valor que le permita al INPEC tomar decisiones enfocadas en la mejora de su gestión en la contratación pública.
* Estadísticas relevantes para el entendimiento de la situación contractual actual.
* Información que presente los comportamientos históricos en la contratación pública del INPEC.

In [2]:
# Importacion de librerias
# Tratamiento de datos
# ==========================================
import re
import time
from datetime import datetime
import pandas as pd
import numpy as np
import data_describe as dd
from string import digits
from gensim.utils import simple_preprocess
from gensim.corpora.textcorpus import strip_multiple_whitespaces
# Graficas
# ==========================================
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
# API REST
# ==========================================
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

import requests

import warnings
warnings.filterwarnings("ignore")

### Importamos Data - SECOP II

In [3]:
# Define the API endpoint URL
url = "https://www.datos.gov.co/resource/jbjy-vk9h.json?nombre_entidad=INSTITUTO NACIONAL PENITENCIARIO Y CARCELARIO INPEC&$limit=1000000"

# Send an HTTP GET request to the API endpoint
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    print("Data received from API:")
else:
    print(f"Request failed with status code {response.status_code}")

Data received from API:


In [4]:
# Set the display option to show all columns
pd.set_option('display.max_columns', None)
secop_II = pd.DataFrame(data=data)
secop_II.head(3)

,nombre_entidad,nit_entidad,departamento,ciudad,localizaci_n,orden,sector,rama,entidad_centralizada,proceso_de_compra,id_contrato,referencia_del_contrato,estado_contrato,codigo_de_categoria_principal,descripcion_del_proceso,tipo_de_contrato,modalidad_de_contratacion,justificacion_modalidad_de,fecha_de_firma,fecha_de_inicio_del_contrato,fecha_de_fin_del_contrato,condiciones_de_entrega,tipodocproveedor,documento_proveedor,proveedor_adjudicado,es_grupo,es_pyme,habilita_pago_adelantado,liquidaci_n,obligaci_n_ambiental,obligaciones_postconsumo,reversion,valor_del_contrato,valor_de_pago_adelantado,valor_facturado,valor_pendiente_de_pago,valor_pagado,valor_amortizado,valor_pendiente_de,valor_pendiente_de_ejecucion,estado_bpin,c_digo_bpin,anno_bpin,saldo_cdp,saldo_vigencia,espostconflicto,urlproceso,destino_gasto,origen_de_los_recursos,dias_adicionados,puntos_del_acuerdo,pilares_del_acuerdo,nombre_representante_legal,nacionalidad_representante_legal,tipo_de_identificaci_n_representante_legal,identificaci_n_representante_legal,g_nero_representante_legal,presupuesto_general_de_la_nacion_pgn,sistema_general_de_participaciones,sistema_general_de_regal_as,recursos_propios_alcald_as_gobernaciones_y_resguardos_ind_genas_,recursos_de_credito,recursos_propios,ultima_actualizacion,codigo_entidad,codigo_proveedor,objeto_del_contrato,fecha_de_inicio_de_ejecucion,fecha_de_fin_de_ejecucion,fecha_inicio_liquidacion,fecha_fin_liquidacion
0,INSTITUTO NACIONAL PENITENCIARIO Y CARCELARIO ...,800215546,Distrito Capital de Bogotá,Bogotá,"Colombia, Bogotá, Bogotá",Nacional,No aplica/No pertenece,Corporación Autónoma,Descentralizada,CO1.BDOS.1994824,CO1.PCCNTR.2537658,DIR GENERAL 065 DE 2021,En ejecución,V1.80111601,PRESTAR SUS SERVICIOS COMO AUXILIAR ADMINISTRA...,Prestación de servicios,Contratación directa,ServiciosProfesionales,2021-05-25T09:05:01.000,2021-05-26T00:00:00.000,2021-12-31T00:00:00.000,No Definido,Cédula de Ciudadanía,1045751778,SERGIO ANDRES INFANZON CASTRO,No,No,No,No,No,No,No,13516000,0,0,13516000,0,0,0,13516000,Válido,No Definido,2019,14880000,0,No,{'url': 'https://community.secop.gov.co/Public...,Funcionamiento,Distribuido,0,No aplica,No aplica,SERGIO ANDRES INFANZON CASTRO,colombiana,Cédula de Ciudadanía,1045751778,Hombre,13516000,0,0,0,0,0,2021-07-02T00:00:00.000,700674096,713779106,PRESTAR SUS SERVICIOS COMO AUXILIAR ADMINISTRA...,NaN,NaN,NaN,NaN
1,INSTITUTO NACIONAL PENITENCIARIO Y CARCELARIO ...,800215546,Distrito Capital de Bogotá,Bogotá,"Colombia, Bogotá, Bogotá",Nacional,No aplica/No pertenece,Corporación Autónoma,Descentralizada,CO1.BDOS.1120466,CO1.PCCNTR.1388926,041 DE 2020,Activo,V1.93151507,PRESTAR POR SUS PROPIOS MEDIOS CON PLENA AUTON...,Prestación de servicios,Contratación directa,ServiciosProfesionales,2020-02-19T10:02:31.000,2020-02-11T00:00:00.000,2020-12-31T00:00:00.000,Como acordado previamente,Cédula de Ciudadanía,7311814,Juan Pablo Martinez Martinez,No,No,No,No,No,No,No,19800000,0,0,19800000,0,0,0,19800000,Válido,No Definido,N/D,19800000,0,No,{'url': 'https://community.secop.gov.co/Public...,Funcionamiento,Distribuido,0,No aplica,No aplica,Juan Pablo Martinez Martinez,Colombiano,Sin Descripcion,7311814,No Definido,0,0,0,0,0,0,2020-02-19T00:00:00.000,700674096,708913900,PRESTAR POR SUS PROPIOS MEDIOS CON PLENA AUTON...,2020-02-11T00:00:00.000,2020-12-31T00:00:00.000,NaN,NaN
2,INSTITUTO NACIONAL PENITENCIARIO Y CARCELARIO ...,800215546,Distrito Capital de Bogotá,Bogotá,"Colombia, Bogotá, Bogotá",Nacional,No aplica/No pertenece,Corporación Autónoma,Descentralizada,CO1.BDOS.1135355,CO1.PCCNTR.1407861,045 DE 2020,terminado,V1.93121607,PRESTAR POR SUS PROPIOS MEDIOS CON PLENA AUTON...,Prestación de servicios,Contratación directa,ServiciosProfesionales,2020-02-27T09:02:59.000,2020-02-21T00:00:00.000,2020-12-31T00:00:00.000,Como acordado previamente,Cédula de Ciudadanía,1030558512,Nydya Alejandra Mendoza Muñoz,No,No,No,No,No,No,No,22000000,0,4933333,17066667,4933333,0,0,22000000,Válido,No Definido,2020,22000000,0,No,{'ur

#### Verificamos Información 

1. `estado_del_proceso`

In [5]:
pd.DataFrame(secop_II['estado_contrato'].value_counts())

,count
estado_contrato,
En ejecución,259
Activo,211
Modificado,173
terminado,101
Borrador,27
Cerrado,14
enviado Proveedor,11
cedido,4
En aprobación,3


2. `fecha_de_firma`

In [7]:
# Convert the 'dates' column to datetime format
secop_II['fecha_de_firma'] = pd.to_datetime(secop_II['fecha_de_firma'])
# Create a new column 'year' by extracting the year from the 'dates' column
secop_II['anno_firma_contrato'] = secop_II['fecha_de_firma'].dt.year
#secop_II['anno_firma_contrato'] = secop_II['anno_firma_contrato'].astype(int)
# Extraigo los contratos por año
pd.DataFrame(secop_II['anno_firma_contrato'].value_counts())

,count
anno_firma_contrato,
2022.0,159
2020.0,144
2021.0,130
2018.0,99
2017.0,96
2019.0,71
2023.0,39
2016.0,26


#### Tipo de cada columna

A continuación podemos observar cada tipo de columna y la memoria que utiliza el proceso.

In [8]:
# Tipo de cada columna
# ==============================================================================
# En pandas, el tipo "object" hace referencia a strings
# datos.dtypes
secop_II.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 805 entries, 0 to 804
Data columns (total 72 columns):
 #   Column                                                            Non-Null Count  Dtype         
---  ------                                                            --------------  -----         
 0   nombre_entidad                                                    805 non-null    object        
 1   nit_entidad                                                       805 non-null    object        
 2   departamento                                                      805 non-null    object        
 3   ciudad                                                            805 non-null    object        
 4   localizaci_n                                                      805 non-null    object        
 5   orden                                                             805 non-null    object        
 6   sector                                                            805 non-

#### Número de observaciones y valores ausentes

Junto con el estudio del tipo de variables, es básico conocer el número de observaciones disponibles y si todas ellas están completas. Los valores ausentes son muy importantes a la hora de crear modelos, la mayoría de algoritmos no aceptan observaciones incompletas o bien se ven muy influenciados por ellas.

In [9]:
# Dimensiones del dataset
# ==============================================================================
print("Número de filas:", secop_II.shape[0])
print("Número de columnas:", secop_II.shape[1])

Número de filas: 805
Número de columnas: 72


In [10]:
#pd.set_option('display.max_rows', None)
# Número de datos ausentes por variable
# ==============================================================================
pd.DataFrame(secop_II.isna().sum().sort_values(ascending=False), columns=["Cantidad de Valores Nulos"]).head(13)

,Cantidad de Valores Nulos
fecha_fin_liquidacion,790
fecha_inicio_liquidacion,790
fecha_de_inicio_de_ejecucion,446
fecha_de_fin_de_ejecucion,446
anno_firma_contrato,41
fecha_de_firma,41
fecha_de_inicio_del_contrato,35
objeto_del_contrato,23
fecha_de_fin_del_contrato,15
valor_pendiente_de_ejecucion,3


En el siguiente dataframe podemos visualizar la cantidad de valores nulos por columna. 
Se observa que lo siguente:
* La columna de `fecha_fin_liquidacion` tiene el % de los valores como NULL.
* La columna de `fecha_inicio_liquidacion` tiene el % de los valores como NULL.
* La columna de `fecha_de_inicio_de_ejecucion` tiene el % de los valores como NULL.
* La columna de `fecha_de_fin_de_ejecucion` tiene el % de los valores como NULL.
* La columna de `anno_firma_contrato` tiene el % de los valores como NULL.
* La columna de `fecha_de_firma` tiene el % de los valores como NULL.
* La columna de `fecha_de_inicio_del_contrato` tiene el % de los valores como NULL.
* La columna de `objeto_del_contrato` tiene el % de los valores como NULL.
* La columna de `fecha_de_fin_del_contrato` tiene el % de los valores como NULL.
* La columna de `valor_pendiente_de_ejecucion` tiene el % de los valores como NULL.
* La columna de `ultima_actualizacion` tiene el % de los valores como NULL.

#### Remover columnas innecesarias

Para tener un mejor panorama de los datos se realizo una entrevista a un contratista vigente del estado (Mario Rivera), el cual asesoro acerca de contratación publica y determino las variables mas importantes a tener en cuenta para la detección de patrones de contratación, adiciones y ejecuciones que ayude con el objetivo de  ofrecer información de valor al INPEC para la toma de decisiones que impacte positivamente en la administración de su contratación pública.

A continuación se eliminaran las columnas que no sean representativas para el requerimiento del cliente.

In [11]:
secop_II_filter = secop_II[['rama','entidad_centralizada','proceso_de_compra','id_contrato','estado_contrato',
 'descripcion_del_proceso','tipo_de_contrato','modalidad_de_contratacion','justificacion_modalidad_de',
 'tipodocproveedor','documento_proveedor','proveedor_adjudicado','es_grupo','liquidaci_n','obligaciones_postconsumo','reversion','valor_del_contrato',
 'valor_de_pago_adelantado','valor_facturado','valor_pendiente_de_pago','valor_pagado','valor_pendiente_de_ejecucion',
 'saldo_cdp', 'urlproceso','destino_gasto','dias_adicionados','nombre_representante_legal','tipo_de_identificaci_n_representante_legal',
 'identificaci_n_representante_legal','objeto_del_contrato']]

secop_II_filter.head(3)

,rama,entidad_centralizada,proceso_de_compra,id_contrato,estado_contrato,descripcion_del_proceso,tipo_de_contrato,modalidad_de_contratacion,justificacion_modalidad_de,tipodocproveedor,documento_proveedor,proveedor_adjudicado,es_grupo,liquidaci_n,obligaciones_postconsumo,reversion,valor_del_contrato,valor_de_pago_adelantado,valor_facturado,valor_pendiente_de_pago,valor_pagado,valor_pendiente_de_ejecucion,saldo_cdp,urlproceso,destino_gasto,dias_adicionados,nombre_representante_legal,tipo_de_identificaci_n_representante_legal,identificaci_n_representante_legal,objeto_del_contrato
0,Corporación Autónoma,Descentralizada,CO1.BDOS.1994824,CO1.PCCNTR.2537658,En ejecución,PRESTAR SUS SERVICIOS COMO AUXILIAR ADMINISTRA...,Prestación de servicios,Contratación directa,ServiciosProfesionales,Cédula de Ciudadanía,1045751778,SERGIO ANDRES INFANZON CASTRO,No,No,No,No,13516000,0,0,13516000,0,13516000,14880000,{'url': 'https://community.secop.gov.co/Public...,Funcionamiento,0,SERGIO ANDRES INFANZON CASTRO,Cédula de Ciudadanía,1045751778,PRESTAR SUS SERVICIOS COMO AUXILIAR ADMINISTRA...
1,Corporación Autónoma,Descentralizada,CO1.BDOS.1120466,CO1.PCCNTR.1388926,Activo,PRESTAR POR SUS PROPIOS MEDIOS CON PLENA AUTON...,Prestación de servicios,Contratación directa,ServiciosProfesionales,Cédula de Ciudadanía,7311814,Juan Pablo Martinez Martinez,No,No,No,No,19800000,0,0,19800000,0,19800000,19800000,{'url': 'https://community.secop.gov.co/Public...,Funcionamiento,0,Juan Pablo Martinez Martinez,Sin Descripcion,7311814,PRESTAR POR SUS PROPIOS MEDIOS CON PLENA AUTON...
2,Corporación Autónoma,Descentralizada,CO1.BDOS.1135355,CO1.PCCNTR.1407861,terminado,PRESTAR POR SUS PROPIOS MEDIOS CON PLENA AUTON...,Prestación de servicios,Contratación directa,ServiciosProfesionales,Cédula de Ciudadanía,1030558512,Nydya Alejandra Mendoza Muñoz,No,No,No,No,22000000,0,4933333,17066667,4933333,22000000,22000000,{'url': 'https://community.secop.gov.co/Public...,Funcionamiento,0,Nydya Alejandra Mendoza Muñoz,Sin Descripcion,1030558512,PRESTAR POR SUS PROPIOS MEDIOS CON PLENA AUTON...


### Normalización de los datos

* Elimino .(puntuación) de la columna de `proceso_de_compra` y `id_contrato`

In [12]:
# Utilizar la función replace con una expresión regular para eliminar los puntos de las columnas
secop_II_filter_norm = secop_II_filter.copy()
secop_II_filter_norm[['proceso_de_compra','id_contrato']] = secop_II_filter_norm[['proceso_de_compra','id_contrato']].replace('\.', '', regex=True)
secop_II_filter_norm.head(3)

,rama,entidad_centralizada,proceso_de_compra,id_contrato,estado_contrato,descripcion_del_proceso,tipo_de_contrato,modalidad_de_contratacion,justificacion_modalidad_de,tipodocproveedor,documento_proveedor,proveedor_adjudicado,es_grupo,liquidaci_n,obligaciones_postconsumo,reversion,valor_del_contrato,valor_de_pago_adelantado,valor_facturado,valor_pendiente_de_pago,valor_pagado,valor_pendiente_de_ejecucion,saldo_cdp,urlproceso,destino_gasto,dias_adicionados,nombre_representante_legal,tipo_de_identificaci_n_representante_legal,identificaci_n_representante_legal,objeto_del_contrato
0,Corporación Autónoma,Descentralizada,CO1BDOS1994824,CO1PCCNTR2537658,En ejecución,PRESTAR SUS SERVICIOS COMO AUXILIAR ADMINISTRA...,Prestación de servicios,Contratación directa,ServiciosProfesionales,Cédula de Ciudadanía,1045751778,SERGIO ANDRES INFANZON CASTRO,No,No,No,No,13516000,0,0,13516000,0,13516000,14880000,{'url': 'https://community.secop.gov.co/Public...,Funcionamiento,0,SERGIO ANDRES INFANZON CASTRO,Cédula de Ciudadanía,1045751778,PRESTAR SUS SERVICIOS COMO AUXILIAR ADMINISTRA...
1,Corporación Autónoma,Descentralizada,CO1BDOS1120466,CO1PCCNTR1388926,Activo,PRESTAR POR SUS PROPIOS MEDIOS CON PLENA AUTON...,Prestación de servicios,Contratación directa,ServiciosProfesionales,Cédula de Ciudadanía,7311814,Juan Pablo Martinez Martinez,No,No,No,No,19800000,0,0,19800000,0,19800000,19800000,{'url': 'https://community.secop.gov.co/Public...,Funcionamiento,0,Juan Pablo Martinez Martinez,Sin Descripcion,7311814,PRESTAR POR SUS PROPIOS MEDIOS CON PLENA AUTON...
2,Corporación Autónoma,Descentralizada,CO1BDOS1135355,CO1PCCNTR1407861,terminado,PRESTAR POR SUS PROPIOS MEDIOS CON PLENA AUTON...,Prestación de servicios,Contratación directa,ServiciosProfesionales,Cédula de Ciudadanía,1030558512,Nydya Alejandra Mendoza Muñoz,No,No,No,No,22000000,0,4933333,17066667,4933333,22000000,22000000,{'url': 'https://community.secop.gov.co/Public...,Funcionamiento,0,Nydya Alejandra Mendoza Muñoz,Sin Descripcion,1030558512,PRESTAR POR SUS PROPIOS MEDIOS CON PLENA AUTON...


* Se define la columna de PRÓRROGADO

In [13]:
# Función para convertir el dataframe 
secop_II_filter_norm['dias_adicionados'] = secop_II_filter_norm['dias_adicionados'].astype(int)
secop_II_filter_norm['prorrogado'] = np.where(secop_II_filter_norm['dias_adicionados'] != 0, 1, 0)

* Se Limpia y normaliza Texto en la data

In [14]:
def norm_text(texto:str):
    word_tokens = simple_preprocess(texto, deacc=True, min_len=2, max_len=40)
    texto = ' '.join(word_tokens)
    return texto

# Convert type str
secop_II_filter_norm[['rama', 'entidad_centralizada']] = secop_II_filter_norm[['rama', 'entidad_centralizada']].astype(str)
secop_II_filter_norm[['estado_contrato', 'tipo_de_contrato']] = secop_II_filter_norm[['estado_contrato', 'tipo_de_contrato']].astype(str)
secop_II_filter_norm[['modalidad_de_contratacion', 'justificacion_modalidad_de']] = secop_II_filter_norm[['modalidad_de_contratacion', 'justificacion_modalidad_de']].astype(str)
secop_II_filter_norm[['tipodocproveedor', 'proveedor_adjudicado']] = secop_II_filter_norm[['tipodocproveedor', 'proveedor_adjudicado']].astype(str)
secop_II_filter_norm[['destino_gasto', 'nombre_representante_legal','tipo_de_identificaci_n_representante_legal']] = secop_II_filter_norm[['destino_gasto', 'nombre_representante_legal','tipo_de_identificaci_n_representante_legal']].astype(str)

# Apply funtion
secop_II_filter_norm[['rama', 'entidad_centralizada']] = secop_II_filter_norm[['rama', 'entidad_centralizada']].apply(lambda x: x.apply(norm_text))
secop_II_filter_norm[['estado_contrato', 'tipo_de_contrato']] = secop_II_filter_norm[['estado_contrato', 'tipo_de_contrato']].apply(lambda x: x.apply(norm_text))
secop_II_filter_norm[['modalidad_de_contratacion', 'justificacion_modalidad_de']] = secop_II_filter_norm[['modalidad_de_contratacion', 'justificacion_modalidad_de']].apply(lambda x: x.apply(norm_text))
secop_II_filter_norm[['tipodocproveedor', 'proveedor_adjudicado']] = secop_II_filter_norm[['tipodocproveedor', 'proveedor_adjudicado']].apply(lambda x: x.apply(norm_text))
secop_II_filter_norm[['destino_gasto', 'nombre_representante_legal','tipo_de_identificaci_n_representante_legal']] = secop_II_filter_norm[['destino_gasto', 'nombre_representante_legal','tipo_de_identificaci_n_representante_legal']].apply(lambda x: x.apply(norm_text))

secop_II_filter_norm.head(2)

,rama,entidad_centralizada,proceso_de_compra,id_contrato,estado_contrato,descripcion_del_proceso,tipo_de_contrato,modalidad_de_contratacion,justificacion_modalidad_de,tipodocproveedor,documento_proveedor,proveedor_adjudicado,es_grupo,liquidaci_n,obligaciones_postconsumo,reversion,valor_del_contrato,valor_de_pago_adelantado,valor_facturado,valor_pendiente_de_pago,valor_pagado,valor_pendiente_de_ejecucion,saldo_cdp,urlproceso,destino_gasto,dias_adicionados,nombre_representante_legal,tipo_de_identificaci_n_representante_legal,identificaci_n_representante_legal,objeto_del_contrato,prorrogado
0,corporacion autonoma,descentralizada,CO1BDOS1994824,CO1PCCNTR2537658,en ejecucion,PRESTAR SUS SERVICIOS COMO AUXILIAR ADMINISTRA...,prestacion de servicios,contratacion directa,serviciosprofesionales,cedula de ciudadania,1045751778,sergio andres infanzon castro,No,No,No,No,13516000,0,0,13516000,0,13516000,14880000,{'url': 'https://community.secop.gov.co/Public...,funcionamiento,0,sergio andres infanzon castro,cedula de ciudadania,1045751778,PRESTAR SUS SERVICIOS COMO AUXILIAR ADMINISTRA...,0
1,corporacion autonoma,descentralizada,CO1BDOS1120466,CO1PCCNTR1388926,activo,PRESTAR POR SUS PROPIOS MEDIOS CON PLENA AUTON...,prestacion de servicios,contratacion directa,serviciosprofesionales,cedula de ciudadania,7311814,juan pablo martinez martinez,No,No,No,No,19800000,0,0,19800000,0,19800000,19800000,{'url': 'https://community.secop.gov.co/Public...,funcionamiento,0,juan pablo martinez martinez,sin descripcion,7311814,PRESTAR POR SUS PROPIOS MEDIOS CON PLENA AUTON...,0


* Limpio descripciones de proceso, dejando solo palabras claves

In [15]:
def clean_text(texto: str):
    # Convert text to lowercase
    texto = texto.lower()
    texto = texto.replace('\n', ' ') # remove line breaks
    texto = re.sub('\S*@\S*\s?', '', texto)  #remove emails 
    texto = re.sub(r'[0-9]+', '', texto) #remove numbers
    texto = strip_multiple_whitespaces(texto) #remove spaces
    word_tokens = simple_preprocess(texto, deacc=True, min_len=4, max_len=15)
    texto = ' '.join(word_tokens)
    return texto

process_text = lambda x: clean_text(x)
# Aplicamos la normalización de texto para descripción del proceso
secop_II_filter_norm['descripcion_del_proceso'] = secop_II_filter_norm['descripcion_del_proceso'].apply(process_text)
# Aplicamos la normalización de texto para objeto del contrato
secop_II_filter_norm['objeto_del_contrato']= secop_II_filter_norm['objeto_del_contrato'].astype(str)
secop_II_filter_norm['objeto_del_contrato'] = secop_II_filter_norm['objeto_del_contrato'].apply(process_text)
secop_II_filter_norm.head(2)

,rama,entidad_centralizada,proceso_de_compra,id_contrato,estado_contrato,descripcion_del_proceso,tipo_de_contrato,modalidad_de_contratacion,justificacion_modalidad_de,tipodocproveedor,documento_proveedor,proveedor_adjudicado,es_grupo,liquidaci_n,obligaciones_postconsumo,reversion,valor_del_contrato,valor_de_pago_adelantado,valor_facturado,valor_pendiente_de_pago,valor_pagado,valor_pendiente_de_ejecucion,saldo_cdp,urlproceso,destino_gasto,dias_adicionados,nombre_representante_legal,tipo_de_identificaci_n_representante_legal,identificaci_n_representante_legal,objeto_del_contrato,prorrogado
0,corporacion autonoma,descentralizada,CO1BDOS1994824,CO1PCCNTR2537658,en ejecucion,prestar servicios como auxiliar administrativo...,prestacion de servicios,contratacion directa,serviciosprofesionales,cedula de ciudadania,1045751778,sergio andres infanzon castro,No,No,No,No,13516000,0,0,13516000,0,13516000,14880000,{'url': 'https://community.secop.gov.co/Public...,funcionamiento,0,sergio andres infanzon castro,cedula de ciudadania,1045751778,prestar servicios como auxiliar administrativo...,0
1,corporacion autonoma,descentralizada,CO1BDOS1120466,CO1PCCNTR1388926,activo,prestar propios medios plena autonomia tecnica...,prestacion de servicios,contratacion directa,serviciosprofesionales,cedula de ciudadania,7311814,juan pablo martinez martinez,No,No,No,No,19800000,0,0,19800000,0,19800000,19800000,{'url': 'https://community.secop.gov.co/Public...,funcionamiento,0,juan pablo martinez martinez,sin descripcion,7311814,prestar propios medios plena autonomia tecnica...,0


### Relleno de data incompleta

Por medio de tecnica scramble html se puede acceder a la Hoja del proceso de cada contrato con la finalidad de obtener información faltante de la base de datos y asi poder completar información de valor para el analisis.

Esta base de datos define estos datos faltante como valore ``No Definido``, la idea es rellenar estos valores con los extraidos de la tecnica de scramble

In [16]:
def found_miss_data(dataframe:pd.DataFrame, valor_especifico:str):
    # Establecer el valor específico que deseas encontrar
    valor_especifico = valor_especifico
    # Utilizar el método 'isin()' para encontrar las columnas que contienen el valor específico
    columnas_con_valor_especifico = dataframe.isin([valor_especifico]).any()
    # Filtrar las columnas que contienen el valor específico
    columnas_filtradas = columnas_con_valor_especifico[columnas_con_valor_especifico].index.tolist()
    # Contar la cantidad de veces que se repite el valor específico en cada columna
    specific_value_counts = pd.DataFrame(dataframe.apply(lambda x: (x == valor_especifico).sum()), columns=["count"])

    return specific_value_counts, columnas_filtradas

specific_value_counts, columnas_filtradas = found_miss_data(secop_II_filter_norm, 'no definido')
print("\nNúmero de columnas con valores [No Definido]:",len(columnas_filtradas))
specific_value_counts[specific_value_counts["count"] != 0].sort_values(by=['count'])


Número de columnas con valores [No Definido]: 2


,count
destino_gasto,6
tipodocproveedor,443


In [17]:
specific_value_counts, columnas_filtradas = found_miss_data(secop_II_filter_norm, 'No Definido')
print("\nNúmero de columnas con valores [No Definido]:",len(columnas_filtradas))
specific_value_counts[specific_value_counts["count"] != 0].sort_values(by=['count'])


Número de columnas con valores [No Definido]: 1


,count
documento_proveedor,17


In [18]:
specific_value_counts, columnas_filtradas = found_miss_data(secop_II_filter_norm, 'Sin Descripcion')
print("\nNúmero de columnas con valores [No Definido]:",len(columnas_filtradas))
specific_value_counts[specific_value_counts["count"] != 0].sort_values(by=['count'])


Número de columnas con valores [No Definido]: 1


,count
identificaci_n_representante_legal,108


Estas son las columnas que hacen falta por datos y que a continuación se extraeran los datos en la hoja de detalle del proceso con la finalidad de completar la información

In [19]:
# Dataframe con data No Definida
secop_II_filter_norm_scra = secop_II_filter_norm.copy()
secop_II_filter_norm_scra.head(3)

,rama,entidad_centralizada,proceso_de_compra,id_contrato,estado_contrato,descripcion_del_proceso,tipo_de_contrato,modalidad_de_contratacion,justificacion_modalidad_de,tipodocproveedor,documento_proveedor,proveedor_adjudicado,es_grupo,liquidaci_n,obligaciones_postconsumo,reversion,valor_del_contrato,valor_de_pago_adelantado,valor_facturado,valor_pendiente_de_pago,valor_pagado,valor_pendiente_de_ejecucion,saldo_cdp,urlproceso,destino_gasto,dias_adicionados,nombre_representante_legal,tipo_de_identificaci_n_representante_legal,identificaci_n_representante_legal,objeto_del_contrato,prorrogado
0,corporacion autonoma,descentralizada,CO1BDOS1994824,CO1PCCNTR2537658,en ejecucion,prestar servicios como auxiliar administrativo...,prestacion de servicios,contratacion directa,serviciosprofesionales,cedula de ciudadania,1045751778,sergio andres infanzon castro,No,No,No,No,13516000,0,0,13516000,0,13516000,14880000,{'url': 'https://community.secop.gov.co/Public...,funcionamiento,0,sergio andres infanzon castro,cedula de ciudadania,1045751778,prestar servicios como auxiliar administrativo...,0
1,corporacion autonoma,descentralizada,CO1BDOS1120466,CO1PCCNTR1388926,activo,prestar propios medios plena autonomia tecnica...,prestacion de servicios,contratacion directa,serviciosprofesionales,cedula de ciudadania,7311814,juan pablo martinez martinez,No,No,No,No,19800000,0,0,19800000,0,19800000,19800000,{'url': 'https://community.secop.gov.co/Public...,funcionamiento,0,juan pablo martinez martinez,sin descripcion,7311814,prestar propios medios plena autonomia tecnica...,0
2,corporacion autonoma,descentralizada,CO1BDOS1135355,CO1PCCNTR1407861,terminado,prestar propios medios plena autonomia tecnica...,prestacion de servicios,contratacion directa,serviciosprofesionales,cedula de ciudadania,1030558512,nydya alejandra mendoza munoz,No,No,No,No,22000000,0,4933333,17066667,4933333,22000000,22000000,{'url': 'https://community.secop.gov.co/Public...,funcionamiento,0,nydya alejandra mendoza munoz,sin descripcion,1030558512,prestar propios medios plena autonomia tecnica...,0


In [20]:
def extract_info_cuantia_url_2(url:str, columna:str):
    ruta_chromedriver = ChromeDriverManager(path='./chromedriver').install()
    s = Service(ruta_chromedriver)
    driver = webdriver.Chrome(service=s)
    driver.get(url=url)

    # Add this line to wait for the user to solve the CAPTCHA manually
    input("Please solve the CAPTCHA and press Enter to continue...")

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    paragraphs = soup.find_all('tr')
    result = []

    if columna == "Destinación del gasto":
        # Find the specific tr element with the desired id
        row = soup.find('tr', {'id': 'trExpenseTypeRow'})

        # Iterate through the cells (td) in the row
        for cell in row.find_all('td'):
            print(cell.text)
            result.append(cell.text)

    driver.quit()
    return result

In [21]:
result = extract_info_cuantia_url_2(url=secop_II_filter_norm_scra['urlproceso'][0]['url'], 
                                    columna="Destinación del gasto") 

[WDM] - Downloading: 100%|██████████| 8.79M/8.79M [00:03<00:00, 3.06MB/s]


Destinación del gasto 
Funcionamiento
 


* Filtramos los datos de la columna ``destino_gasto=='No Definido'`` y asignamos el valor real obtenido de la Hoja del detalle del proceso de los contratos obtenida de la url de la columna de ``urlproceso``

In [22]:
filtro_destino_gasto = secop_II_filter_norm_scra[secop_II_filter_norm_scra['destino_gasto']=='no definido']
print(f"Filas:{filtro_destino_gasto.shape[0]}\nColumnas:{filtro_destino_gasto.shape[1]}")

Filas:6
Columnas:31


In [23]:
index_list = []
for index, row in filtro_destino_gasto.iterrows():
    print("Se corrigio el index:",index)
    index_list.append(index)
    url = row['urlproceso']['url']
    result = extract_info_cuantia_url_2(url=url, columna="Destinación del gasto")
    if len(result) != 0:
        secop_II_filter_norm_scra.loc[index, 'destino_gasto'] = result[1]
    else:
        secop_II_filter_norm_scra.loc[index, 'destino_gasto'] = 'No encontrado en Hoja Proceso'

Se corrigio el index: 18
Destinación del gasto 
Funcionamiento
 
Se corrigio el index: 174
Destinación del gasto 
Funcionamiento
 
Se corrigio el index: 448
Destinación del gasto 

 
Se corrigio el index: 487
Destinación del gasto 
Funcionamiento
 
Se corrigio el index: 551
Destinación del gasto 
Funcionamiento
 
Se corrigio el index: 643
Destinación del gasto 
Funcionamiento
 


In [24]:
secop_II_filter_norm_scra.loc[448,"destino_gasto"]='No encontrado en Hoja Proceso'
secop_II_filter_norm_scra.loc[index_list,"destino_gasto"]

18                    Funcionamiento
174                   Funcionamiento
448    No encontrado en Hoja Proceso
487                   Funcionamiento
551                   Funcionamiento
643                   Funcionamiento
Name: destino_gasto, dtype: object

* Filtramos los datos de la columna `NAN` y asignamos el valor real obtenido de la Hoja del detalle del proceso de los contratos obtenida de la url de la columna de ``urlproceso``

In [25]:
pd.DataFrame(secop_II_filter_norm_scra.isna().sum().sort_values(ascending=False), columns=["Cantidad de Valores Nulos"]).head(3)

,Cantidad de Valores Nulos
valor_pendiente_de_ejecucion,3
rama,0
valor_del_contrato,0


1. El valor de `valor_pendiente_de_ejecucion` con triple 0101 debido a que no fue posible encontrar estos valores en la hoja de detalle del proceso

In [26]:
secop_II_filter_norm_scra.loc[secop_II_filter_norm_scra['valor_pendiente_de_ejecucion'].isnull(), "valor_pendiente_de_ejecucion"]= 10101
secop_II_filter_norm_scra[secop_II_filter_norm_scra["valor_pendiente_de_ejecucion"]==10101]


,rama,entidad_centralizada,proceso_de_compra,id_contrato,estado_contrato,descripcion_del_proceso,tipo_de_contrato,modalidad_de_contratacion,justificacion_modalidad_de,tipodocproveedor,documento_proveedor,proveedor_adjudicado,es_grupo,liquidaci_n,obligaciones_postconsumo,reversion,valor_del_contrato,valor_de_pago_adelantado,valor_facturado,valor_pendiente_de_pago,valor_pagado,valor_pendiente_de_ejecucion,saldo_cdp,urlproceso,destino_gasto,dias_adicionados,nombre_representante_legal,tipo_de_identificaci_n_representante_legal,identificaci_n_representante_legal,objeto_del_contrato,prorrogado
269,corporacion autonoma,descentralizada,CO1BDOS596506,CO1PCCNTR678914,activo,contratar adquisicion repuestos para equipos c...,compraventa,minima cuantia,presupuesto inferior al de la menor cuantia,no definido,830007379,comsistelco sas,No,No,No,No,61252243,0,0,0,0,10101,61252243,{'url': 'https://community.secop.gov.co/Public...,funcionamiento,0,eliberto olivares guzman,sin descripcion,Sin Descripcion,contratar adquisicion repuestos para equipos c...,0
343,corporacion autonoma,descentralizada,CO1BDOS596682,CO1PCCNTR678773,modificado,contratar adquisicion repuestos para circuito ...,compraventa,minima cuantia,presupuesto inferior al de la menor cuantia,no definido,890205950,comunisander sas,No,No,No,No,14606382,0,0,0,0,10101,20000000,{'url': 'https://community.secop.gov.co/Public...,funcionamiento,0,sergio montanez serrano,sin descripcion,91476533,contratar adquisicion repuestos para circuito ...,0
711,corporacion autonoma,descentralizada,Desconocido,CO1PCCNTR680030,activo,descripcion,no especificado,contratacion directa,no especificado,cedula de ciudadania,1126178205,carlos camilo rodriguez,No,No,No,No,6291000,0,0,0,0,10101,6291000,{'url': 'https://community.secop.gov.co/Public...,funcionamiento,0,carlos camilo rodriguez cano,sin descripcion,1126178205,prestar propios medios plena autonomia tecnica...,0


In [27]:
#secop_II_filter_norm_scra.loc[secop_II_filter_norm_scra['fecha_de_inicio_del_contrato'].isnull(), "urlproceso"].to_list()

#### Definición DTYPES

Se define un tipo de dato para cada columna

In [28]:
secop_II_filter_norm_scra.dtypes

rama                                          object
entidad_centralizada                          object
proceso_de_compra                             object
id_contrato                                   object
estado_contrato                               object
descripcion_del_proceso                       object
tipo_de_contrato                              object
modalidad_de_contratacion                     object
justificacion_modalidad_de                    object
tipodocproveedor                              object
documento_proveedor                           object
proveedor_adjudicado                          object
es_grupo                                      object
liquidaci_n                                   object
obligaciones_postconsumo                      object
reversion                                     object
valor_del_contrato                            object
valor_de_pago_adelantado                      object
valor_facturado                               

* Defino las columnas tipo INTEGER

In [29]:
secop_II_filter_norm_scra.head(2)

,rama,entidad_centralizada,proceso_de_compra,id_contrato,estado_contrato,descripcion_del_proceso,tipo_de_contrato,modalidad_de_contratacion,justificacion_modalidad_de,tipodocproveedor,documento_proveedor,proveedor_adjudicado,es_grupo,liquidaci_n,obligaciones_postconsumo,reversion,valor_del_contrato,valor_de_pago_adelantado,valor_facturado,valor_pendiente_de_pago,valor_pagado,valor_pendiente_de_ejecucion,saldo_cdp,urlproceso,destino_gasto,dias_adicionados,nombre_representante_legal,tipo_de_identificaci_n_representante_legal,identificaci_n_representante_legal,objeto_del_contrato,prorrogado
0,corporacion autonoma,descentralizada,CO1BDOS1994824,CO1PCCNTR2537658,en ejecucion,prestar servicios como auxiliar administrativo...,prestacion de servicios,contratacion directa,serviciosprofesionales,cedula de ciudadania,1045751778,sergio andres infanzon castro,No,No,No,No,13516000,0,0,13516000,0,13516000,14880000,{'url': 'https://community.secop.gov.co/Public...,funcionamiento,0,sergio andres infanzon castro,cedula de ciudadania,1045751778,prestar servicios como auxiliar administrativo...,0
1,corporacion autonoma,descentralizada,CO1BDOS1120466,CO1PCCNTR1388926,activo,prestar propios medios plena autonomia tecnica...,prestacion de servicios,contratacion directa,serviciosprofesionales,cedula de ciudadania,7311814,juan pablo martinez martinez,No,No,No,No,19800000,0,0,19800000,0,19800000,19800000,{'url': 'https://community.secop.gov.co/Public...,funcionamiento,0,juan pablo martinez martinez,sin descripcion,7311814,prestar propios medios plena autonomia tecnica...,0


In [30]:
secop_II_filter_norm_scra[['valor_del_contrato',
                      'valor_de_pago_adelantado','valor_facturado',
                      'valor_pendiente_de_pago','valor_pagado',
                      'valor_pendiente_de_ejecucion','saldo_cdp','dias_adicionados']]=secop_II_filter_norm_scra[['valor_del_contrato',
                      'valor_de_pago_adelantado','valor_facturado',
                      'valor_pendiente_de_pago','valor_pagado',
                      'valor_pendiente_de_ejecucion','saldo_cdp','dias_adicionados']].astype(int)

In [31]:
secop_II_filter_norm_scra.to_csv('data/extrac_secopII_I.csv', index=False)